In [ ]:
# IDE - Google Colab
# Do this before start, 
!pip install selenium
!apt-get update
!apt-get install chromium-chromedriver
!pip install bs4
!pip install selenium
!pip install requests
!pip install re

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import csv
from selenium import webdriver
import re
import pandas as pd


def find_association(soup):
  association = soup.find('th')
  return association.text

def acronym_generator(soup):
  acronym = soup[0]
  for i in range(1, len(soup)):
    if soup[i-1] == ' ':
      acronym += soup[i]
  acronym = acronym.upper()
  return acronym

def find_name(soup):
  name = soup.find_all('tr')
  if len(name) == 2:
    names = name[1].text.split()
    return names[1]
  else:
    names = name[2].text.split()
    return names[1]

def find_confederation(soup):
  confederation = soup.find('tr').find_all('td')[1]
  return confederation.text

def find_website(soup):
    try:
      website = soup.find('a', attrs = {'target': '_blank'})
      return website.text
    except:
      return ''
def find_email(soup):
  try:
    email = soup.find('a')
    return email.text
  except:
    return ''

def find_president(soup):
  try:  
    president = soup.find_all('td')[1]
    if president.text == 'Affiliated to :':
      return ''
    return president.text
  except:
    return ''

def find_sports_director(soup):
  try:
    director = soup.find_all('td')[1]
    if director.text == 'Affiliated to :':
      return ''
    return director.text
  except:
    return ''

def find_office(soup):
  try:
    office = soup.find_all('td')[1]
    if len(office.text) > 36:
      return office.text.strip()
    else:
      return office.text
  except:
    return ''

def find_contact(soup):
  try:
    contact = soup.find_all('td')[3]
    contact_person = soup.find_all('td')[2]
    if contact_person.text == 'Contact person:':
      return contact.text
    else:
      return ''
  except:
    return ''

def find_zipcode(soup):
  try:    
    zipcode = soup.find_all('td')[7]
    zipcode_check = soup.find_all('td')[6]
    if zipcode_check.text == 'Zip code / City:':
      return zipcode.text
    else:
      return ''
  except:
    return ''

def find_address(soup):
  try:
    address = soup.find_all('td')[5]
    address_check = soup.find_all('td')[4]
    if address_check.text == 'Address:':
      return address.text
    else:
      return ''
  except:
    return ''

def find_phone(soup):
  try:
    phone = soup.find_all('td')[11]
    phone_check = soup.find_all('td')[10]
    if phone_check.text == 'Phone:':
      return phone.text
    else: 
      return ''
  except: 
    return ''

def main():

  my_list = []
  
  #Chrome_Driver
  userAgent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('user-agent= {}'.format(userAgent))
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options = options)

  url = 'https://www.eurobillard.org/pages/federations-31.html'

  driver.get(url)
  selenium_get = driver.page_source
  soup = bs(selenium_get, 'html.parser')
  
  info = soup.find('div', class_ = 'c_35 list_fede col')
  find_url = info.find_all('div')
  nmr = 0
  
  
  my_list = []
  for x in find_url:
    regex_url = re.compile(r'\/federations\/[a-zA-Z0-9-.]+')
    country_url = regex_url.findall(str(find_url))  
    while nmr <=26:
      for i in country_url:
        my_dict = {}
        nmr = nmr + 1
        if nmr == 27:
          break
        url = 'https://www.eurobillard.org{}'.format(i)
        driver.get(url)
        selenium_get = driver.page_source
        soup = bs(selenium_get, 'html.parser')
        
        # INFOS
        info = soup.find_all('tbody')[0]
        info2 = soup.find_all('table')[1]
        info3 = soup.find_all('table')[2]
        try:  
          info4 = soup.find_all('table')[3]
        except:
          pass
        info5 = soup.find('thead')
        
        
        my_dict['Federation'] = find_association(info5)
        my_dict['Acronym'] = acronym_generator(find_association(info5))
        my_dict['Name'] = find_name(info)
        my_dict['Confederation'] = find_confederation(info)
        my_dict['Website'] = find_website(info)
        my_dict['Email'] = find_email(info2)
        my_dict['President'] = find_president(info3)
        try:  
          my_dict['Director'] = find_sports_director(info4)
        except:
          my_dict['Director'] = ''
        my_dict['Office'] = find_office(info2)
        my_dict['Contact'] = find_contact(info2)
        my_dict['Zipcode'] = find_zipcode(info2)
        my_dict['Address'] = find_address(info2)
        my_dict['Phone'] = find_phone(info2)

        my_dict['Sport'] = 'Billard'
        my_dict['Reach'] = 'National'
        my_dict['Category'] = 'Recognized'
        my_dict['Affilated'] = 'Confédération Européenne de Billard'
        my_dict['Recorded By'] = 'Eduardo Savio'
        my_dict['Recorded Date'] = '24/02/2022'

        my_list.append(my_dict)
      return my_list
        
        

res = main()

#save as xlsx
df = pd.DataFrame(data=res)
df.to_excel('Confédération Européenne de Billard.xlsx')